In [0]:
spark.conf.set("fs.azure.account.auth.type.dlsassign.dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.dlsassign.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.dlsassign.dfs.core.windows.net", "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2023-08-31T03:26:51Z&st=2023-08-07T19:26:51Z&spr=https&sig=wU3ruQRItsr%2BNfdVcUerP4mqT2WHtA5QYOrPWZEGPwA%3D")

In [0]:
import unittest
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException

class TestDataTransfer(unittest.TestCase):    
    def test_data_transfer(self):
        """
        Test the data transfer logic using Spark DataFrames.

        This method performs the following steps:
        1. Reads data from the source container 'demo-data' and displays it.
        2. Tries to read existing data from the destination container 'raw'. If it fails, assumes no existing data.
        3. Computes the incremental DataFrame based on whether existing data is available.
        4. Checks if the incremental DataFrame contains any rows and prints corresponding messages.
        """
        # Reads data from container demo-data and display it
        source_data_df = spark.read.csv('abfs://demo-data@dlsassign.dfs.core.windows.net/Customers.csv', header=True)
        
        # Read existing data from the destination container if available
        try:
            existing_df = spark.read.parquet('abfs://raw@dlsassign.dfs.core.windows.net/customers.parquet')
        except AnalysisException:
            existing_df = None
        
        # Your data transfer logic
        if existing_df is not None:
            incremental_df = existing_df.exceptAll(source_data_df)
        else:
            incremental_df = source_data_df
        
        # Verify if the DataFrame is empty or not using count()
        if incremental_df.head(1):
            print("Bucket is empty")
            # Here, you can write logic to save the DataFrame or any other verification
        else:
            print("Bucket is not empty")
            
if __name__ == "__main__":
    test_runner = unittest.main(argv=[''], exit=False)
    test_runner.result.printErrors()


/usr/lib/python3.9/socket.py:775: ResourceWarning: unclosed <socket.socket fd=59, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 49894), raddr=('127.0.0.1', 46129)>
  self._sock = None
.

Bucket is not empty



----------------------------------------------------------------------
Ran 1 test in 36.970s

OK

